In [7]:
import re

from glob import glob
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

pd.set_option("display.max_rows", 300)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

## e149 の valid を確認する

In [2]:
ls ../checkpoints/e149/best

dataset-metadata.json
fold_0_epoch_1_1.52047_-1.00000_0.72111_checkpoint.pth
fold_1_epoch_4_1.58928_-1.00000_0.71914_checkpoint.pth
fold_2_epoch_3_1.54997_-1.00000_0.71985_checkpoint.pth
fold_3_epoch_3_1.52912_-1.00000_0.72332_checkpoint.pth
fold_4_epoch_3_1.53969_-1.00000_0.72253_checkpoint.pth


In [3]:
trn_df = pd.read_csv('../inputs/origin/train.csv')
display(trn_df.shape, trn_df.head())

(27481, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
trn_df = trn_df[trn_df.selected_text.notnull()]
trn_df.shape

(27480, 4)

In [5]:
ckpts = []
for fckpt in glob('../checkpoints/e149/best/*.pth'):
    ckpts.append(torch.load(fckpt))

In [119]:
# def _get_predicted_texts(input_ids, y_preds_head, y_preds_tail, tokenizer):
#     predicted_texts = []
#     for input_id, y_pred_head, y_pred_tail \
#             in zip(input_ids, y_preds_head, y_preds_tail):
#         pred_label_head = y_pred_head.argmax()
#         pred_label_tail = y_pred_tail.argmax()
#         predicted_text = tokenizer.decode(
#             input_id[pred_label_head:pred_label_tail])
#         predicted_texts.append(predicted_text)
# 
#     return predicted_texts
import re

def _get_predicted_texts_offsets(texts, offsets_list, sentiments,
                                 y_preds_head, y_preds_tail,
                                 neutral_origin=True,
                                 head_tail_equal_handle='tail',
                                 pospro={'magic': False, 'magic_2': True, 'magic_3': False},
                                 tail_index='kernel'):
    predicted_texts = []
    for text, offsets, sentiment, y_pred_head, y_pred_tail \
            in zip(texts, offsets_list, sentiments,
                   y_preds_head, y_preds_tail):
        if neutral_origin and sentiment == 'neutral' \
                or len(text.split()) < 2:
            predicted_texts.append(text)
            continue
        text1 = ' ' + ' '.join(text.split())
        pred_label_head = y_pred_head.argmax()
        pred_label_tail = y_pred_tail.argmax()

        if tail_index == 'kernel':
            pred_label_tail += 1

        # NOTE: change here from kernel
        if pred_label_head >= pred_label_tail:
            if head_tail_equal_handle == 'nothing':
                raise NotImplementedError()
            elif head_tail_equal_handle == 'head':
                pred_label_tail = pred_label_head + 1
            elif head_tail_equal_handle == 'tail':
                pred_label_head = pred_label_tail - 1
            elif head_tail_equal_handle == 'larger':
                while pred_label_head >= pred_label_tail:
                    print(f'flip found, {text[:10]}...')
                    if y_pred_head.max() <= y_pred_tail.max():
                        # NOTE: copy にしたい
                        y_pred_head[y_pred_head.argmax()] = 0.
                    else:
                        y_pred_tail[y_pred_tail.argmax()] = 0.
                    pred_label_head = y_pred_head.argmax()
                    pred_label_tail = y_pred_tail.argmax()
                    if tail_index == 'kernel':
                        pred_label_tail += 1
            elif head_tail_equal_handle == 'larger_2':
                if y_pred_head.max() <= y_pred_tail.max():
                    y_pred_head = y_pred_tail - 1
                else:
                    y_pred_tail = y_pred_head + 1
            else:
                raise NotImplementedError()

        ss = offsets[pred_label_head][0]
        ee = offsets[pred_label_tail - 1][1] + 1
        predicted_text = text1[ss:ee].strip()
        if pospro['magic']:
                ee -= text[ss:ee].strip().count('   ')
                ee += text[ss:ee].strip().count('  ')
                if '  ' in text[:(ss+ee) // 2]:
                    predicted_text = text[ss:ee].strip() 
        if pospro['magic_2']:
            if '  ' in text[:ee]:
                temp = predicted_text
                predicted_text = repr(predicted_text)[1:-1]
                predicted_text = re.sub(r'\\', r'\\', predicted_text)
                predicted_text = re.sub('\|', '\|', predicted_text)
                predicted_text = re.sub('\.', '\.', predicted_text)
                predicted_text = re.sub('\*', '\*', predicted_text)
                predicted_text = re.sub('\+', '\+', predicted_text)
                predicted_text = re.sub('\?', '\?', predicted_text)
                predicted_text = re.sub('\$', '\$', predicted_text)
                predicted_text = re.sub('\^', '\^', predicted_text)
                predicted_text = re.sub('\(', '\(', predicted_text)
                predicted_text = re.sub('\)', '\)', predicted_text)
                predicted_text = re.sub('\{', '\{', predicted_text)
                predicted_text = re.sub('\}', '\}', predicted_text)
                predicted_text = re.sub('\[', '\[', predicted_text)
                predicted_text = re.sub('\]', '\]', predicted_text)
                query = re.sub(' ', ' +', predicted_text)
                matched_predicted_texts = re.findall(query, text)
                if len(matched_predicted_texts) > 1:
                    # warning
                    print('--------------------------------------------------------')
                    print(f'more than 1 mathced_predicted_texts: {matched_predicted_texts}')
                    print(temp)
                    print(query)
                    print(f'ss: {ss} -- ee: {ee} -- pred_label_head: {pred_label_head} -- pred_label_tail: {pred_label_tail}')
                    print(f'text1: {text1}')
                    print(offsets)
                matched_predicted_text = matched_predicted_texts[0]
                ss = text.find(matched_predicted_text)
                if ss == -1:
                    raise Exception('did not find magic 2 index')
                ee = ss + len(predicted_text)
                # 先頭の空白分後退
                if text[0] == ' ':
                    ss -= 1
                # ee までにある２つ以上の +alpha 空白数分後退
                for cnt_base in re.findall("[$^ ] +[^ ]", text[:ee]):
                    ss -= cnt_base[1:].count(' ')
                ss = max(ss, 0)
                predicted_text = text[ss:ee].strip()
        if pospro['magic_3']:
            ss = text.find(predicted_text)
            if ss == -1:
                raise Exception('did not find magic 2 index')
            ee = ss + len(predicted_text)
            # 先頭の空白分後退
            if text[0] == ' ':
                ss -= 1
            # ee までにある２つ以上の +alpha 空白数分後退
            for cnt_base in re.findall("[$^ ] +[^ ]", text[:ee]):
                ss -= cnt_base[1:].count(' ')
            ss = max(ss, 0)
            predicted_text = text[ss:ee].strip()

        predicted_texts.append(predicted_text)

    return predicted_texts

In [8]:
def _get_selected_texts(input_ids, labels_head, labels_tail, tokenizer):
    predicted_texts = []
    for input_id, pred_label_head, pred_label_tail \
            in zip(input_ids, labels_head, labels_tail):
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [9]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [80]:
trn_df = trn_df.reset_index()
trn_df = trn_df.set_index('textID')
for ckpt in tqdm(ckpts):
    texts = []
    offsets = []
    sentiments = []
    for textID in tqdm(ckpt['val_textIDs']):
        text = trn_df.loc[textID].text
        tweet_offsets = tokenizer.encode(' '.join(text.split())).offsets
        offsets.append([(0, 0)] * 4 + tweet_offsets + [(0, 0)])
        texts.append(text)
        sentiment = trn_df.loc[textID].sentiment
        sentiments.append(sentiment)
    ckpt['texts'] = texts
    ckpt['offsets'] = offsets
    ckpt['sentiments'] = sentiments
trn_df = trn_df.reset_index()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [120]:
# %debug
for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    predicted_text = _get_predicted_texts_offsets(ckpt['texts'], ckpt['offsets'], ckpt['sentiments'], ckpt['val_preds'][0],  ckpt['val_preds'][1], pospro={'magic': True, 'magic_2': False, 'magic_3': False})
    trn_df.loc[ckpt['val_textIDs'], 'predicted_texts'] = predicted_text
    trn_df = trn_df.reset_index()
trn_df

,textID,level_0,index,text,selected_text,sentiment,predicted_texts,jaccard
0,cb774db0d1,0,0,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going",1.000000
1,549e992a42,1,1,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,o SAD,0.333333
2,088c60f138,2,2,my boss is bullying me...,bullying me,negative,s bullying me...,0.250000
3,9642c003ef,3,3,what interview! leave me alone,leave me alone,negative,! leave me alone,0.750000
4,358bd9e861,4,4,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****,",1.000000
...,...,...,...,...,...,...,...,...
27475,4eac33d1c0,27475,27476,wish we could come see u on Denver husband l...,d lost,negative,nd lost his job and can`t afford i,0.125000
27476,4f4c4fc327,27476,27477,I`ve wondered about rake to. The client has ...,", don`t force",negative,"y, don`t forc",0.500000
27477,f67aae2310,27477,27478,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,y good,0.142857
27478,ed167662a5,27478,27479,But it was worth it ****.,But it was worth it ****.,positive,s worth,0.166667


In [115]:
from tools.metrics import jaccard

In [122]:
trn_df['jaccard'] = trn_df.apply(lambda row: jaccard(row['predicted_texts'], row['selected_text']), axis=1)
trn_df['jaccard'].mean()

0.6423486607647163

In [85]:
predicted_text = 'e its bomb!'
text = 'Eating some ice cream cake  its bomb!'
query = re.sub(' ', ' +', predicted_text)
res = re.findall(query, text)
res

['e  its bomb!']

In [31]:
query

'e +its +bomb!'

In [40]:
re.findall("e +its +bomb!", text)

['e  its bomb!']

In [78]:
tokenizer.encode(' missing jmichelle... told you treat me like a step child! smh').offsets

[(0, 8),
 (8, 10),
 (10, 13),
 (13, 18),
 (18, 21),
 (21, 26),
 (26, 30),
 (30, 36),
 (36, 39),
 (39, 44),
 (44, 46),
 (46, 51),
 (51, 57),
 (57, 58),
 (58, 61),
 (61, 62)]

In [20]:
for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    manual_selected_texts = _get_selected_texts(ckpt['val_input_ids'], ckpt['val_labels'][0],  ckpt['val_labels'][1], tokenizer)
    trn_df.loc[ckpt['val_textIDs'], 'manual_selected_text'] = manual_selected_texts
    trn_df = trn_df.reset_index()
trn_df

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",True,"i`d have responded, if i were going","i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,False,sooo sad,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me,False,bullying,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,False,leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",False,"sons of ****,","sons of ****,"
...,...,...,...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish we could come see u on Denver husband los...,d lost,False,lost his job and can`t afford it,husband lost
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,I`ve wondered about rake to. The client has ma...,", don`t force",False,don`t force,", don`t force"
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,Yay good for both of you. Enjoy the break - yo...,Yay good for both of you.,False,yay good,yay good for both of you.
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,But it was worth it ****.,But it was worth it ****.,True,worth,but it was worth it ****.


In [21]:
trn_df['selected_text_lower'] = trn_df['selected_text'].apply(lambda x: " " + " ".join(x.lower().split()))
trn_df

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",True,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,False,sooo sad,sooo sad,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me,False,bullying,bullying me,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,False,leave me alone,leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",False,"sons of ****,","sons of ****,","sons of ****,"
...,...,...,...,...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish we could come see u on Denver husband los...,d lost,False,lost his job and can`t afford it,husband lost,d lost
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,I`ve wondered about rake to. The client has ma...,", don`t force",False,don`t force,", don`t force",", don`t force"
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,Yay good for both of you. Enjoy the break - yo...,Yay good for both of you.,False,yay good,yay good for both of you.,yay good for both of you.
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,But it was worth it ****.,But it was worth it ****.,True,worth,but it was worth it ****.,but it was worth it ****.


In [22]:
trn_df['manual_and_selected_intersection_len'] = trn_df.apply(lambda row: len(set(row['manual_selected_text'].split()) & set(row['selected_text_lower'].split())), axis=1)
trn_df['manual_and_selected_intersection_len'].value_counts().sort_index()

0      334
1     7211
2     2313
3     2163
4     1983
5     1603
6     1380
7     1176
8     1012
9      879
10     812
11     732
12     669
13     578
14     558
15     504
16     470
17     454
18     435
19     395
20     414
21     335
22     317
23     289
24     211
25     114
26      77
27      40
28      14
29       8
Name: manual_and_selected_intersection_len, dtype: int64

In [23]:
trn_df['manual_and_selected_tokenized_intersection_len'] = trn_df.apply(lambda row: len(set(tokenizer.encode(' '.join(row['manual_selected_text'].split()), add_special_tokens=False).ids) & set(tokenizer.encode(' '.join(row['selected_text_lower'].split()), add_special_tokens=False).ids)), axis=1)
trn_df['manual_and_selected_tokenized_intersection_len'].value_counts().sort_index()

0       66
1     5172
2     2825
3     1859
4     1746
5     1628
6     1335
7     1152
8     1027
9      961
10     800
11     701
12     684
13     649
14     584
15     502
16     514
17     477
18     445
19     452
20     380
21     361
22     375
23     354
24     368
25     317
26     307
27     278
28     243
29     238
30     165
31     128
32     134
33      88
34      56
35      29
36      25
37      16
38      15
39       8
40       8
41       5
42       3
Name: manual_and_selected_tokenized_intersection_len, dtype: int64

In [24]:
trn_df.to_csv('../inputs/nes_info/e080_dataset_trn_df.csv', index=False)

## error analysis

In [2]:
trn_df = pd.read_csv('../inputs/nes_info/e080_dataset_trn_df.csv').dropna()
display(trn_df.shape, trn_df.head())

(27423, 12)

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",True,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going",7,9
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,False,sooo sad,sooo sad,sooo sad,2,3
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me,False,bullying,bullying me,bullying me,2,2
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,False,leave me alone,leave me alone,leave me alone,3,3
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",False,"sons of ****,","sons of ****,","sons of ****,",3,4


In [ ]:
tokenizer.encode("------------------------------------------------------------").tokens

In [ ]:
tokenizer.decode([48364])

In [ ]:
tokenizer.decode(tokenizer.encode("lovelovelove<3").ids[:-2])

In [62]:
tokenizer.encode(' '.join('...awesome'.split())).tokens

['Ġ...', 'aw', 'esome']

In [63]:
trn_df.query('selected_text_lower != manual_selected_text').sort_values('jaccard').sample(100, random_state=72)[['sentiment', 'text', 'selected_text_lower', 'manual_selected_text', 'predicted_texts', 'jaccard']]

,sentiment,text,selected_text_lower,manual_selected_text,predicted_texts,jaccard
5384,negative,"hopping in the shower, you can help me tidy ...",y my room its a **** hol,tidy my room its a **** hole,"**** hole,",0.125000
24593,positive,have you check oceanup?'miley cyrus justin ga...,m.im with you whatever happen,mmm.im with you whatever happen,you ok with justin?hmmm.im with you whatever ...,0.571429
21781,positive,so I`ve organised a day of indulgence & luxur...,e likes the surprise..,she likes the surprise..,a pamper day fingers crossed she likes the su...,0.300000
25289,negative,hey! chutti was very tiring.. and have to tra...,tiring.,tiring..,tiring..,0.000000
3871,positive,hey yall were amazing at bamboozle! thanks f...,e amazing,were amazing,amazing,0.500000
2226,negative,1000 = make sure it has the word poo or balls...,.. i gave up on it,... i gave up on it,suck,0.000000
2180,negative,Trying to go to sleep but no luck I think i`m...,but no luck i think i`m sick.,but no luck i think i`m sick...,sick...,0.000000
1587,positive,WOOOOOOOOOO are you coming to Nottingham at...,t? lovelovelove,point? lovelovelove,lovelovelove<3,0.000000
17172,negative,What happened? I`m afraid I missed a large pa...,? i`m afraid i missed,? i`m afraid i missed,i`m afraid,0.400000
4705,positive,Gud nite all...will catch you all again tomorrow,gud nite all..,gud nite all...,gud,0.333333


In [81]:
trn_df.text.str.contains('----------').sum()

0